# Оптимизация графа вычислений: слияние Conv–BatchNorm–ReLU

## 1. Введение

Во время оптимизации графа вычислений фреймворки, такие как ONNX Runtime, TensorRT или OpenVINO, объединяют последовательные операции свёртки (Conv), нормализации (BatchNorm) и активации (ReLU) в одну эквивалентную, но более эффективную операцию.  

Основная причина этого — уменьшение количества обращений к памяти и запусков отдельных вычислительных ядер на GPU или CPU.  
Главное математическое основание: и свёртка, и нормализация представляют собой **линейные преобразования**, которые могут быть объединены в одно эквивалентное линейное преобразование.

---

## 2. Свёрточная операция (Convolution)

Свёртка в свёрточной нейронной сети — это операция линейного преобразования, вычисляемая по формуле

$$
Y[i,j] = \sum_{u=0}^{H_W-1} \sum_{v=0}^{W_W-1} W[u,v] \cdot X[i+u, j+v] + b
$$

где  
- \( X \) — входное изображение,  
- \( W \) — фильтр (ядро свёртки),  
- \( b \) — смещение (bias),  
- \( Y \) — выход свёртки,  
- \( H_W, W_W \) — высота и ширина фильтра.

### 2.1. Параметры свёртки

- **Stride (шаг свёртки)** — насколько ядро сдвигается по входу.  
- **Padding** — добавление рамки нулей вокруг входа.  
- **Valid padding** — свёртка без добавления нулей (padding = 0).  
- **Same padding** — паддинг подбирается так, чтобы выход имел те же размеры, что и вход.

Для valid-свёртки (padding = 0) размер выхода равен:

$$
H_{out} = H_{in} - H_W + 1, \quad
W_{out} = W_{in} - W_W + 1
$$

---

## 3. Batch Normalization

Batch Normalization в режиме обучения нормализует активации внутри батча, а затем масштабирует и сдвигает их с помощью обучаемых параметров \( \gamma \) и \( \beta \):

$$
\text{BN}(y) = \gamma \frac{y - \mu_{\text{batch}}}{\sqrt{\sigma^2_{\text{batch}} + \epsilon}} + \beta
$$

- \( \mu_{\text{batch}} \) — среднее по батчу,  
- \( \sigma^2_{\text{batch}} \) — дисперсия по батчу,  
- \( \epsilon \) — стабилизирующая константа,  
- \( \gamma, \beta \) — обучаемые параметры.

### 3.1. BatchNorm на inference

На этапе inference используются накопленные статистики (running mean и running variance):

$$
\text{BN}(y) = \gamma \frac{y - \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta
$$

Это линейная операция над каждым элементом \( y \), которую можно записать в виде

$$
\text{BN}(y) = a y + c,
$$

где

$$
a = \frac{\gamma}{\sqrt{\sigma^2 + \epsilon}}, \quad c = \beta - a \mu.
$$

---

## 4. Слияние Conv и BatchNorm

Пусть свёртка вычисляет

$$
Y = W * X + b
$$

а BatchNorm применяет

$$
\text{BN}(Y) = aY + c.
$$

Подставим выражение для \( Y \) в формулу BatchNorm:

$$
\text{BN}(W * X + b) = a(W * X + b) + c = (aW) * X + (ab + c).
$$

Таким образом, можно определить новые параметры свёртки:

$$
W' = aW, \quad b' = ab + c.
$$

Теперь новая свёртка

$$
Y' = W' * X + b'
$$

эквивалентна последовательному применению Conv и BatchNorm.

---

### 4.1. Числовой пример

Рассмотрим минимальный случай:  
вход \( X \in \mathbb{R}^{3 \times 3} \), ядро \( W \in \mathbb{R}^{2 \times 2} \), bias \( b = 0 \).

$$
X =
\begin{bmatrix}
1 & 2 & 0\\
0 & 1 & 3\\
2 & 2 & 1
\end{bmatrix},
\quad
W =
\begin{bmatrix}
1 & 0\\
-1 & 2
\end{bmatrix}.
$$

Вычисляем valid-свёртку (stride=1, padding=0):

$$
Y =
\begin{bmatrix}
3 & 7\\
2 & 1
\end{bmatrix}.
$$

Теперь применим BatchNorm с параметрами  
\( \gamma = 3, \beta = -1, \mu = 2, \sigma^2 = 4, \epsilon = 0 \).

Сначала вычислим коэффициенты:

$$
a = \frac{3}{\sqrt{4}} = 1.5, \quad c = \beta - a\mu = -1 - 1.5\cdot2 = -4.
$$

BatchNorm выполняет:

$$
\text{BN}(Y) = 1.5Y - 4 =
\begin{bmatrix}
0.5 & 6.5\\
-1 & -2.5
\end{bmatrix}.
$$

Теперь пересчитаем параметры свёртки:

$$
W' = 1.5W =
\begin{bmatrix}
1.5 & 0\\
-1.5 & 3
\end{bmatrix},
\quad
b' = 1.5\cdot0 - 4 = -4.
$$

Новая свёртка:

$$
Y' = W' * X + b'
$$

даёт тот же результат:

$$
Y' =
\begin{bmatrix}
0.5 & 6.5\\
-1 & -2.5
\end{bmatrix}
= \text{BN}(Y).
$$

Таким образом, операции Conv и BatchNorm полностью эквивалентны одной свёртке с параметрами \( W', b' \).

---

## 5. Добавление ReLU

Функция активации ReLU определяется как

$$
\text{ReLU}(x) = \max(0, x).
$$

Хотя ReLU не является линейной операцией, она элемент-wise и не изменяет размерность выходного тензора.  
Поэтому backend-системы могут включить её непосредственно в тот же вычислительный kernel, выполняя операцию активации сразу после свёртки без записи промежуточных данных в память.

Результатом становится **fused-kernel** (например, `ConvBnRelu`), выполняющий все три операции за один проход.

---

## 6. Оптимизация графа в ONNX Runtime и TensorRT

Современные backend-системы выполняют оптимизацию графа вычислений в несколько этапов.

1. **Поиск паттернов**  
   Граф анализируется на наличие последовательностей узлов Conv → BatchNorm → ReLU.

2. **Пересчёт параметров**  
   Для каждой найденной последовательности вычисляются:
   $$
   a = \frac{\gamma}{\sqrt{\sigma^2 + \epsilon}}, \quad c = \beta - a\mu,
   $$
   $$
   W' = aW, \quad b' = ab + c.
   $$

3. **Переписывание графа**  
   Узлы BatchNorm и ReLU удаляются, а Conv заменяется новой операцией с параметрами \( W', b' \).  
   В некоторых системах (например, TensorRT) создаётся специальный узел `FusedConv`.

4. **Преимущества**  
   - Уменьшается количество обращений к памяти.  
   - Сокращается число запусков GPU-ядер.  
   - Снижается накладной overhead синхронизации.  
   - Повышается общая пропускная способность вычислений.

---

## 7. Вывод

Слияние Conv, BatchNorm и ReLU основано на линейности их математических преобразований.  
BatchNorm при inference представляет собой линейную операцию, поэтому её можно объединить с параметрами свёртки:

$$
W' = aW, \quad b' = ab + (\beta - a\mu).
$$

Это позволяет сократить вычислительный граф без потери точности результатов.  
В оптимизированном графе фреймворк выполняет эквивалентную последовательность операций значительно быстрее за счёт уменьшения обращений к памяти и объединения вычислений в единый fused kernel.

